# Schaefer Based Control Loop

In [2]:
import pandas as pd

## The Data

In [3]:
df = pd.read_csv('rocklobster.csv')
df.head()

,year,catch,effort
0,1945,809,232
1,1946,854,253
2,1947,919,289
3,1948,1360,382
4,1949,1872,1046
